<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#FuzzyWuzzy-Package" data-toc-modified-id="FuzzyWuzzy-Package-1">FuzzyWuzzy Package</a></span></li><li><span><a href="#Example" data-toc-modified-id="Example-2">Example</a></span></li><li><span><a href="#Turn-rows-into-single-tags" data-toc-modified-id="Turn-rows-into-single-tags-3">Turn rows into single tags</a></span></li></ul></div>

## FuzzyWuzzy Package

In [2]:
!pip install fuzzywuzzy[speedup]

In [3]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import pandas as pd

In [4]:
tick_comp_df = pd.read_csv("../00-data/company_tickers.csv")
tick_comp_df.head()

,Symbol,Security
0,MMM,3M
1,AOS,A. O. Smith
2,ABT,Abbott
3,ABBV,AbbVie
4,ABMD,Abiomed


In [5]:
article = ["Apple stock went up. Tesla stock dropped. AMZN had good news. Google remains fair. Stryker is a company. Yum! and Best Buy"]
tickers = tick_comp_df["Symbol"].to_list()
companies = tick_comp_df["Security"].to_list()
ticks_comps = tickers + companies
print(ticks_comps)

['MMM', 'AOS', 'ABT', 'ABBV', 'ABMD', 'ACN', 'ATVI', 'ADM', 'ADBE', 'ADP', 'AAP', 'AES', 'AFL', 'A', 'AIG', 'APD', 'AKAM', 'ALK', 'ALB', 'ARE', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AMD', 'AEE', 'AAL', 'AEP', 'AXP', 'AMT', 'AWK', 'AMP', 'ABC', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'ANTM', 'AON', 'APA', 'AAPL', 'AMAT', 'APTV', 'ANET', 'AIZ', 'T', 'ATO', 'ADSK', 'AZO', 'AVB', 'AVY', 'BKR', 'BLL', 'BAC', 'BBWI', 'BAX', 'BDX', 'WRB', 'BRK.B', 'BBY', 'BIO', 'TECH', 'BIIB', 'BLK', 'BK', 'BA', 'BKNG', 'BWA', 'BXP', 'BSX', 'BMY', 'AVGO', 'BR', 'BRO', 'BF.B', 'CHRW', 'CDNS', 'CZR', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CARR', 'CTLT', 'CAT', 'CBOE', 'CBRE', 'CDW', 'CE', 'CNC', 'CNP', 'CDAY', 'CERN', 'CF', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CTXS', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'CL', 'CMCSA', 'CMA', 'CAG', 'COP', 'ED', 'STZ', 'CEG', 'COO', 'CPRT', 'GLW', 'CTVA', 'COST', 'CTRA', 'CCI', 'CSX', 'CMI', 'CVS', '

In [6]:
scores = process.extract(article[0], ticks_comps, scorer=fuzz.token_set_ratio)
scores

[('A', 100), ('AMZN', 100), ('YUM', 100), ('Apple', 100), ('Best Buy', 100)]

In [7]:
tags = [scores[i][0] for i in range(len(scores)) if scores[i][1]>90]
tags

['A', 'AMZN', 'YUM', 'Apple', 'Best Buy']

In [8]:
def tag_article(article, companies, df): #, tickers
    if (article == "NA") | (article==""):
        tags = np.nan
    else:
        scores = process.extract(article, 
                                 companies,
                                 scorer=fuzz.token_set_ratio, 
                                 limit=50)
        comp_tags = [scores[i][0] for i in range(len(scores)) if scores[i][1]>90]
        tags = list(df.loc[df.Security.isin(comp_tags), "Symbol"])
        if len(tags) == 0:
            tags = np.nan
    return(tags)

## Example

In [9]:
from bs4 import BeautifulSoup
import requests
import re

In [10]:
URL = "https://finance.yahoo.com/news/fintech-stocks-fall-amid-report-of-apples-financial-services-plans-201032568.html"
def yahoo_article(URL):
    nf = "Content is currently unavailable"
    page = requests.get(URL,headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'})
    soup = BeautifulSoup(page.content, "html.parser")
    if nf in soup.prettify():
        pass
    try:
        try:
            master = soup.find(id="Masterwrap")
            final = master.find("div", class_="caas-body")
        except AttributeError:
            master = soup.find(id="Masterwrap2Col")
            final = master.find("div", class_="caas-body")
    except AttributeError:
        pass
    text = re.sub('(<p>)', ' ', str(final))
    text = re.sub('<[^>]+>', '', str(text))
    sen_list = text.split(". ")
    article = [".\n ".join(sen_list[:-2])][0] # remove author signature

    return(article)

In [11]:
yahoo_article(URL)

" Block (SQ), PayPal (PYPL), Marqeta (MQ) Visa (V), and other payment companies' shares fell on Wednesday afternoon after Bloomberg reported Apple (AAPL) is working on a plan dubbed “Breakout” aimed at bringing financial services in-house.\n The report states the tech giant is building a payment processing technology and infrastructure as part of an effort to decrease its dependence on outside partners.\n The strategy is aimed at future financial products instead of existing ones.\n Shares of Block, formally known as Square, fell by more than 4% an hour before the closing bell on Wednesday.\n PayPal shares were down about 2%.\n Visa and Global Payments (GNP) was also lower.Payment companies stock price action on March 30, 2022 The reported plan would expand Apple's pretense into the financial services industry.\n The Cupertino, California–based company has been growing its services businesses.\n The tech giant already offers an Apple Card in connection with Goldman Sachs.\n It also run

In [12]:
article = yahoo_article(URL)#"https://news.yahoo.com/red-hot-buccaneers-not-looking-231637664.html")

In [13]:
article

" Block (SQ), PayPal (PYPL), Marqeta (MQ) Visa (V), and other payment companies' shares fell on Wednesday afternoon after Bloomberg reported Apple (AAPL) is working on a plan dubbed “Breakout” aimed at bringing financial services in-house.\n The report states the tech giant is building a payment processing technology and infrastructure as part of an effort to decrease its dependence on outside partners.\n The strategy is aimed at future financial products instead of existing ones.\n Shares of Block, formally known as Square, fell by more than 4% an hour before the closing bell on Wednesday.\n PayPal shares were down about 2%.\n Visa and Global Payments (GNP) was also lower.Payment companies stock price action on March 30, 2022 The reported plan would expand Apple's pretense into the financial services industry.\n The Cupertino, California–based company has been growing its services businesses.\n The tech giant already offers an Apple Card in connection with Goldman Sachs.\n It also run

In [14]:
process.extract(article, 
                companies,
                scorer=fuzz.token_set_ratio, 
                limit=50)

[('Apple', 100),
 ('Global Payments', 100),
 ('Goldman Sachs', 100),
 ('PayPal', 100),
 ('Visa', 100),
 ('PNC Financial Services', 90),
 ('S&P Global', 89),
 ('DXC Technology', 83),
 ('SVB Financial', 82),
 ('TJX Companies', 82),
 ('Air Products', 80),
 ('Lincoln Financial', 69),
 ('AT&T', 67),
 ('Republic Services', 64),
 ('Ameriprise Financial', 62),
 ('Cincinnati Financial', 62),
 ("People's United Financial", 61),
 ('Southern Company', 61),
 ('Estée Lauder Companies', 60),
 ('T. Rowe Price', 59),
 ('U.S. Bank', 55),
 ('United Parcel Service', 50),
 ('Universal Health Services', 48),
 ('Snap-on', 44),
 ('West Pharmaceutical Services', 44),
 ('Hartford (The)', 40),
 ("Lowe's", 29),
 ("Moody's", 25),
 ('Bank of America', 24),
 ("Domino's", 22),
 ('A. O. Smith', 20),
 ("Hershey's", 20),
 ("Kellogg's", 20),
 ("Campbell's", 18),
 ("McDonald's", 18),
 ('Alphabet (Class A)', 12),
 ('Packaging Corporation of America', 12),
 ('Discovery (Series A)', 11),
 ('News Corp (Class A)', 11),
 ('Unde

In [46]:
tag_article(yahoo_article(URL),companies)

['Apple', 'Global Payments', 'Goldman Sachs', 'PayPal', 'Visa']

In [47]:
tag_article(yahoo_article(URL), 
                         ticks_comps)

['A',
 'ARE',
 'AAPL',
 'TECH',
 'IT',
 'HAS',
 'PYPL',
 'V',
 'Apple',
 'Global Payments',
 'Goldman Sachs',
 'PayPal',
 'Visa']

In [116]:
#tags = tag_article(yahoo_article(URL),companies,tick_comp_df)
tags = tag_article("afsdhf", companies, tick_comp_df)
tags

nan

In [117]:
tag_df = pd.DataFrame(columns=["tags"])
tag_df.at[0, 'tags'] = tags
tag_df

,tags
0,NaN


In [118]:
tag_df.at[1, 'tags'] = tags
tag_df

,tags
0,NaN
1,NaN


In [119]:
tag_df.dropna()

,tags


## Turn rows into single tags

In [ ]:
def row_tags(old_df):
    new_df = pd.DataFrame(columns=old_df.columns)
    for i in range(len(new_df)):
        new_tags = new_df.tags[i].strip("][").replace("'","").split(', ')
        for t in new_tags:
            new_df = new_df.append(dff.iloc[i,:-1]).reset_index(drop=True)
            new_df.tags.iloc[-1] = t
    return(new_df)